In [1]:
!pip install datasets
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1


In [2]:
import openai
import os
from datasets import Dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from huggingface_hub import login
import torch
import random
import re
import numpy as np

In [3]:
login()

In [4]:
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [15]:
# @title prompts
prompt_get_part = f'''

## پرسش
{{question}} \n
این یک پرسش پزشکی است برای طبقه بندی پرسش یکی از موضوعات زیر را انتخاب کنید به گونه ای که بیشترین تطابق را با پرسش داشته باشد

Topic List = [
    "Part 1: The Profession of Medicine",
    "Part 2: Cardinal Manifestations and Presentation of Diseases",
    "Part 3: Pharmacology",
    "Part 4: Oncology and Hematology",
    "Part 5: Infectious Diseases",
    "Part 6: Disorders of the Cardiovascular System",
    "Part 7: Disorders of the Respiratory System",
    "Part 8: Critical Care Medicine",
    "Part 9: Disorders of the Kidney and Urinary Tract",
    "Part 10: Disorders of the Gastrointestinal System",
    "Part 11: Immune-Mediated, Inflammatory, and Rheumatologic Disorders",
    "Part 12: Endocrinology and Metabolism",
    "Part 13: Neurologic Disorders",
    "Part 14: Poisoning, Drug Overdose, and Envenomation",
    "Part 15: Disorders Associated with Environmental Exposures",
    "Part 16: Genes, the Environment, and Disease",
    "Part 17: Global Medicine",
    "Part 18: Aging",
    "Part 19: Consultative Medicine",
    "Part 20: Frontiers"
]
به عنوان پاسخ تنها و تنها یک عدد برگردانید به گونه ای که بیشترین تطابق را با پرسش داشته باشد.
'''

prompt_eval = f'''
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n
'''

prompt_eval_bare = f'''
به پرسش زیر به پاسخ بدهید و تنها از زبان فارسی استفاده کنید!
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

\n
'''

prompt_eval_bare_COT = f'''
به پرسش زیر قدم به قدم فکر کنید و زنجیره افکار (chain of thought) خود برای رسیدن به پاسخ را به طور کامل شرح دهید و تنها یک گزینه را انتخاب کنید.
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
'''

prompt_eval_bare_fully = f'''
{{question}} \n
{{choices}}
'''

prompt_eval_bare_fully_with_examples = f'''
{{examples}} \n
{{question}} \n
{{choices}}
'''

prompt_eval_with_examples = f'''
## نمونه ها
{{examples}}

Above are examples for medical Q&A.

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n
'''

prompt_eval_context_bare = f'''
{{context}} \n
{{question}} \n
{{choices}}
'''
prompt_eval_with_context = f'''
## Context
{{context}} \n

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
این یک متن از یک کتاب مرجع است به همراه یک پرسش پزشکی و گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n '''

prompt_eval_with_context_and_examples = f'''
## نمونه ها
{{examples}}
در بالا نمونه هایی از پرسش پاسخ پزشکی آورده شده است.

## متن
{{context}} \n

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n '''

prompt_example = f'''
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

## پاسخ
{{answer}} \n
'''

prompt_extraction = f'''
an answer to a multiple choice question is given to you, your task is
to extract the option that has been chosen,
Please provide the extracted answer in the given format,without any additional explanation
# Format
[answer] e.g. [B]
# Answer
{{answer}}
'''

prompt_critic = f'''
یک پرسش چند گزینه ای پزشکی به همراه پاسخی که یک دانشجو به آن داده است به شما داده شده است
به کمک پاسخ صحیح پرسش که به همراه توضیحاتی پیوست شده است اشکالات پاسخ دانشجو را در فرمت گفته شده به او بگویید بدون آنکه مستقیما اشاره ای به گزینه صحیح بکنید.
در ضمن از توصیه به خواندن منابع نیز بپرهیزید و هر آنچه لازم است را به صورت مستقیم بنویسید.
# پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

## پاسخ صحیح
{{correct_answer}} \n

## توضیحات پاسخ صحیح
{{Expression}} \n

## پاسخ دانشجو
{{student_answer}} \n

#فرمت
1. نقد نخست
2. نقدم دوم
etc...
'''

prompt_refinement = f'''
گزینه ای که انتخاب کردید اشتباه بود با توجه به نقدی که توسط یک متخصص پزشکی برای شما فراهم شده است
دوباره به این پرسش قدم به قدم فکر کنید و زنجیره افکار (chain of thought) خود برای رسیدن به پاسخ را به طور کامل شرح دهید و  گزینه ای دیگر را انتخاب کنید.
همچنین تنها زنجیره افکار خود و پاسخ به پرسش مطرح شده را به عنوان خروجی برگرداندید و از به کار بردن عبارتی همچون "با توجه به نقد ارائه شده" یا "پاسخ اصلاح شده" بپرهیزید

## نقد
{{critic}} \n
'''

In [22]:
# @title utility
# Set openai key if using gpt4o as engine.
#os.environ['OPENAI_API_KEY'] = "OPEN AI KEY HERE"
def extract_samples(task, numShot, model_prompt):
    questions, answer_choices, correct_answers = task_load(task, 'train')
    example_indexes = random.sample(range(len(questions)), numShot)
    example_list = []
    for i in example_indexes:
        example_list.append(model_prompt.format(question=questions[i], choices=format_choices(answer_choices[i]), answer=correct_answers[i]))
    return example_list

def get_ds_from_df(df,task):
    if task == 'qas':
       df['Question'] = df['Question'].astype(str)
       df['Option1'] = df['Option1'].astype(str)
       df['Option2'] = df['Option2'].astype(str)
       df['Option3'] = df['Option3'].astype(str)
       df['Option4'] = df['Option4'].astype(str)
       df['Expression'] = df['Expression'].astype(str)
       df['Correct_answer'] = df['Correct_answer'].astype(str)
       df['choice_type'] = df['choice_type'].astype(str)
       df['subject_name'] = df['subject_name'].astype(str)
       df['topic_name'] = df['topic_name'].astype(str)
       ds = Dataset.from_pandas(df)
       return ds
    else:
       raise Exception("TASK NOT FOUND")

def append_record_to_excel(file_path, Question, question_choices,
                           correct_answer, model_prompt, AI_answer):
    new_record = {
        'Question': Question,
        'question_choices': question_choices,
        'correct_answer': correct_answer,
        'model_prompt':  model_prompt,
        'AI_answer': AI_answer
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)

def format_examples(examples):
    formatted_examples = []
    for row in examples:
        example = f'## Question {row["question"]} \n ## Answer {row["answer"]}'
        formatted_examples.append(example)
    return "\n".join(formatted_examples)

def task_load(task, split):
    if task=="qas":
        df = pd.read_excel(task+'.xlsx')
        ds =get_ds_from_df(df,task)
        questions = [ds[i]['Question'] for i in range(len(ds))]
        answer_choices = [{"A": ds[i]['Option1'], "B": ds[i]['Option2'], "C": ds[i]['Option3'], "D": ds[i]['Option4']} for i in range(len(ds))]
        correct_answers = [chr(int(ds[i]['Correct_answer'])+65) for i in range(len(ds))]
        expressions = [ds[i]['Expression'] for i in range(len(ds))]
        return questions, answer_choices, correct_answers,expressions
    else:
        raise Exception("TASK NOT FOUND")

def filterContext(context):
    end_tag = "</end>"
    if end_tag in context:
        return context.split(end_tag)[0] + end_tag
    return context

def run_inference(messages, engine, temp=1, max_tokens_output=200, tokenizer=None, model=None, local=False):
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda:0')
    with torch.no_grad():
         outputs = model.generate(inputs, max_new_tokens=max_tokens_output, do_sample = True, temperature=temp)
         text = tokenizer.batch_decode(outputs)[0]
         answer = re.split(r'<\|CHATBOT_TOKEN\|>', text)[-1].strip().removesuffix('<|END_OF_TURN_TOKEN|>').strip()
         return answer

def run_inference0(messages, engine, temp=1, max_tokens_output=200, tokenizer=None, model=None, local=False):
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda:0')
    with torch.no_grad():
         outputs = model.generate(inputs, max_new_tokens=max_tokens_output, do_sample = True, temperature=temp)
         text = tokenizer.batch_decode(outputs)[0]
         answer = re.split(r'<\|CHATBOT_TOKEN\|>', text)[-1].strip().removesuffix('<|END_OF_TURN_TOKEN|>').strip()
         return answer

def get_unlocal_AI_answer(prompt,m):
    response = openai.ChatCompletion.create(
        model=m,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0
    )
    result = response['choices'][0]['message']['content'].strip().lower()
    return result

In [8]:
# @title model setting

print("RUNNING NORMAL IMPLEMENTATION")
ENGINE = "CohereForAI/aya-expanse-8b"
SPLIT = "test"
ENGINE_TEMPERATURE = 1.0
MAX_TOKEN_OUTPUT = 1024
TASK = 'qas'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained(ENGINE)
model = AutoModelForCausalLM.from_pretrained(ENGINE,torch_dtype=torch.bfloat16).to(device)
model.eval()
## OUTPUT RUN INFO:
print("Model Running: " + ENGINE)

RUNNING NORMAL IMPLEMENTATION


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model Running: CohereForAI/aya-expanse-8b


In [10]:
# @title Load the test
question_list, answer_choices_list, correct_answer_list,expression_list = task_load(TASK, SPLIT)
print(f"{TASK} loaded succesfully. Now conducting evaluation on {len(question_list)} samples.")

qas loaded succesfully. Now conducting evaluation on 821 samples.


In [27]:
def extract_choice(answer):
    extr = get_unlocal_AI_answer(prompt_extraction.format(answer=answer),'deepseek-chat')
    if extr == '[a]' or  extr == '[A]' or extr == 'a' or extr == 'A' or extr == '[الف]' or extr == 'الف' or extr == '[آ]' or extr == 'آ' or extr == '[ا]' or extr == 'ا':
       return 'A'
    elif extr == '[b]' or extr == '[B]' or extr == 'b' or extr == 'B' or extr == '[ب]' or extr == 'ب':
       return 'B'
    elif extr == '[c]' or extr == '[C]' or extr == 'c' or extr == 'C' or extr == '[ج]' or extr == 'ج':
       return 'C'
    elif extr == '[d]' or extr == '[D]' or extr == 'd' or extr == 'D' or extr == '[د]' or extr == 'د':
       return 'D'
    else:
       return input('answer did not extraxted input manualy\n Answer:' + answer)

def resume_the_communication(question_list, answer_choices_list, correct_answer_list,expression_list,bgn):
    question_list = question_list[bgn:]
    answer_choices_list = answer_choices_list[bgn:]
    correct_answer_list = correct_answer_list[bgn:]
    for i, (question, answer_choices, correct_answer,expression) in tqdm(enumerate(zip(question_list, answer_choices_list,correct_answer_list,expression_list))):
        prompt = prompt_eval_bare_COT
        model_prompt = prompt.format(question=question, choices=format_choices(answer_choices))
        messages = [{"role": "user", "content": f"{model_prompt}"}]
        student_answer = run_inference(messages, ENGINE, ENGINE_TEMPERATURE, MAX_TOKEN_OUTPUT, tokenizer, model, local=True)
        ext = extract_choice(student_answer)
        print(student_answer)
        print(ext)
        print(correct_answer)
        print('-----------------------------------------------')
        if ext == correct_answer:
           continue
        else:
           rejected_answer = student_answer
           messages.append({"role": "assistant", "content": student_answer})
           count = 0
           while count < 10:
                 count += 1
                 p = prompt_critic.format(question=question,
                                          choices=format_choices(answer_choices),
                                          correct_answer=correct_answer,
                                          Expression=expression,
                                          student_answer=student_answer)
                 critic = get_unlocal_AI_answer(p,'deepseek-reasoner')
                 messages.append({"role": "user", "content":prompt_refinement.format(critic=critic)})
                 student_answer = run_inference0(messages, ENGINE, ENGINE_TEMPERATURE, MAX_TOKEN_OUTPUT, tokenizer, model, local=True)
                 messages.append({"role": "assistant", "content": student_answer})
                 ext = extract_choice(student_answer)
                 print(student_answer)
                 print(ext)
                 print(correct_answer)
                 print('-----------------------------------------------')



        file_path = 'answers.xlsx'
        append_record_to_excel(file_path, question, answer_choices,
                           correct_answer, model_prompt, student_answer)

In [28]:
resume_the_communication(question_list, answer_choices_list, correct_answer_list,expression_list,bgn = 0)

0it [00:00, ?it/s]

## زنجیره افکار


برای پاسخ دادن به این سوال، باید درک کنیم که "مانیتور واکسن" به چه چیزی اشاره دارد و ارتباط آن با واکسن OPV چیست. OPV مخفف واکسن فلج اطفال خوراکی است. مانیتور واکسن ابزاری نیست که به طور مستقیم با واکسن ارتباط داشته باشد، بلکه به نظارت بر **واکنش بدن به واکسن**  می پردازد.

مانیتور واکسن معمولاً علائمی مانند  تب، سردرد یا ناراحتی در محل تزریق را ثبت می کند. این علائم ممکن است نشان دهنده پاسخ طبیعی و سالم سیستم ایمنی بدن به واکسن باشد.

اکنون به گزینه های ارائه شده نگاه می کنیم:

* **گزینه [A]**: رنگ دایره بیرونی همانند مربع داخلی است - این گزینه بی معنی است زیرا ارتباطی با واکسن یا مانیتور آن ندارد. 
* **گزینه [B]**: رنگ دایره بیرونی تیره تر از مربع داخلی است - این گزینه نیز ارتباط منطقی با واکسن یا تفسیر نتایج مانیتور ندارد.
* **گزینه [C]**: رنگ دایره بیرونی روشن تر از مربع داخلی است -  این گزینه نیز  بی ارتباط است. مانیتور واکسن به دنبال اندازه گیری علائم فیزیکی است، نه رنگ. 
* **گزینه [D]**: هیچ کدام از موارد فوق -  این گزینه دقیق ترین پاسخ است زیرا هیچ یک از گزینه

0it [04:10, ?it/s]


KeyboardInterrupt: 

In [21]:
text = """<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>به پرسش زیر قدم به قدم فکر کنید و زنجیره افکار (chain of thought) خود برای رسیدن به پاسخ را به طور کامل شرح دهید و تنها یک گزینه را انتخاب کنید.
## پرسش
آیا می‌توان از واکسن opv استفاده کرد اگر مانیتور واکسن نشان دهد؟


## گزینه ها
[A] : رنگ دایره بیرونی همانند مربع داخلی است
[B] : رنگ دایره بیرونی تیره‌تر از مربع داخلی است
[C] : رنگ دایره بیرونی روشن‌تر از مربع داخلی است
[D] : هیچکدام از موارد فوق<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>برای پاسخ به این پرسش، باید درک کنیم که مانیتور واکسن (که معمولاً به عنوان ابزاری برای نظارت بر ایمنی واکسن استفاده می‌شود) چه اطلاعاتی را ارائه می‌دهد و چگونه با واکسن OPV (واکسن ویروس نازل اطفال) ارتباط دارد.

**زنجیره افکار:**

1. **درک واکسن OPV:** واکسن OPV برای پیشگیری از بیماری سیاه سرفه (پولیو) استفاده می‌شود. این واکسن از یک شکل ضعیف یا غیرفعال شده از ویروس پلیو استفاده می‌کند تا سیستم ایمنی بدن را تحریک کند.

2. **نقش مانیتور واکسن:** مانیتور واکسن، معمولاً یک نمودار یا سیستم ثبت داده‌ها است که برای پیگیری واکنش‌های منفی احتمالی پس از واکسیناسیون به کار می‌رود. این مانیتور نشان می‌دهد که آیا واکسن به درستی تزریق شده، عوارض جانبی رخ داده یا نه.

3. **ارتباط با رنگ دایره و مربع:** گزینه‌های ارائه شده در پرسش به رنگ‌های دایره بیرونی و مربع داخلی اشاره دارند. این می‌تواند نمادین باشد و احتمالاً به برخی ویژگی‌های بصری در مانیتور واکسن اشاره دارد.

4. **تحلیل گزینه‌ها:**
   - [A] رنگ دایره بیرونی همانند مربع داخلی است: این می‌تواند نشان‌دهنده تطابق یا هماهنگی باشد، اما به طور مستقیم با ایمنی واکسن OPV مرتبط نیست.
   - [B] رنگ دایره بیرونی تیره‌تر از مربع داخلی است: این تفاوت رنگ ممکن است نشان‌دهنده بروز یک واکنش شدید باشد، اما نمی‌توان به طور مستقیم آن را با واکسن OPV مرتبط دانست.
   - [C] رنگ دایره بیرونی روشن‌تر از مربع داخلی است: این می‌تواند نشان‌دهنده یک واکنش ضعیف یا عدم وجود عارضه جانبی باشد، که در واقع خبر خوبی است و نشان می‌دهد واکسن به درستی کار کرده است.
   - [D] هیچکدام از موارد فوق: با توجه به تحلیل قبلی، گزینه [C] به نظر مناسب‌ترین پاسخ است.

5. **نتیجه‌گیری:** اگر مانیتور واکسن نشان دهد که واکسن OPV با موفقیت تزریق شده و هیچ عارضه جانبی قابل توجهی رخ نداده است، می‌توان از این واکسن استفاده کرد. بنابراین، پاسخ صحیح **[C] رنگ دایره بیرونی روشن‌تر از مربع داخلی است** می‌باشد.<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|USER_TOKEN|>گزینه ای که انتخاب کردید اشتباه بود با توجه به نقدی که توسط یک متخصص پزشکی برای شما فراهم شده است
دوباره به این پرسش قدم به قدم فکر کنید و زنجیره افکار (chain of thought) خود برای رسیدن به پاسخ را به طور کامل شرح دهید و  گزینه ای دیگر را انتخاب کنید.
همچنین تنها زنجیره افکار خود و پاسخ به پرسش مطرح شده را به عنوان خروجی برگرداندید و از به کار بردن عبارتی همچون "با توجه به نقد ارائه شده" یا "پاسخ اصلاح شده" بپرهیزید

## نقد
1. نقد نخست: در تحلیل خود، مانیتور واکسن را به‌اشتباه به عنوان ابزار نظارتی برای عوارض جانبی یا اثربخشی واکسن تفسیر کرده‌اید. این مانیتور در واقع یک نشانگر حرارتی (vvm) است که سلامت واکسن را بر اساس شرایط دمایی ارزیابی می‌کند، نه پیامدهای بالینی آن.

2. نقد دوم: ارتباط رنگ‌ها با ایمنی واکسن opv را نادرست استنباط کرده‌اید. تغییر رنگ دایره بیرونی نسبت به مربع داخلی نشان‌دهنده قرارگیری واکسن در معرض دمای نامناسب است، نه شدت عوارض جانبی یا موفقیت واکسیناسیون.

3. نقد سوم: استدلال شما در گزینه [c] ("رنگ روشن‌تر نشان‌دهنده واکنش ضعیف یا عدم عارضه است") کاملاً معکوس است. در واقع، وقتی دایره بیرونی **تیره‌تر** از مربع داخلی می‌شود، نشان‌دهنده آسیب حرارتی به واکسن است و استفاده از آن را **غیرمجاز** می‌کند.

4. نقد چهارم: نتیجه‌گیری نهایی شما مبتنی بر این فرض اشتباه است که "رنگ روشن‌تر = ایمنی واکسن". این در حالی است که تنها زمانی استفاده از واکسن opv مجاز است که دایره بیرونی **تیره‌تر از مربع داخلی نباشد** (مطابق با دستورالعمل‌های سازمان بهداشت جهانی).

5. نقد پنجم: تمرکز شما بر مفاهیم جانبی مانند "تزریق موفق" یا "عدم عارضه" باعث غفلت از اصل مسئله شده است. پرسش صرفاً به تفسیر صحیح نشانگر حرارتی (vvm) و رابطه رنگ‌های آن محدود می‌شود، نه ارزیابی اثربخشی بالینی واکسن.<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>**زنجیره افکار:**

1. **تفسیر درست مانیتور واکسن:** متخصص پزشکی درست اشاره می‌کند که مانیتور واکسن، مانند یک نشانگر حرارتی (vvm)، سلامت واکسن را بر اساس دمای آن ارزیابی می‌کند، نه اثرات بالینی.

2. **ارتباط دمای واکسن با ایمنی:**  ما باید به یاد داشته باشیم که واکسن OPV باید در دمای مناسب نگهداری و تزریق شود. اگر مانیتور نشان دهد که واکسن در معرض دماهای نامناسب قرار گرفته است (مثلاً دایره بیرونی تیره‌تر از مربع داخلی باشد)، آنگاه واکسن فاسد یا نامناسب برای استفاده است.

3. **تفسیر علائم دمایی:**
   - **دایره بیرونی روشن‌تر از مربع داخلی:** نشان‌دهنده دمای مناسب و سالم بودن واکسن است.
   - **دایره بیرونی تیره‌تر از مربع داخلی:** نشان‌دهنده آسیب حرارتی و نامناسب بودن واکسن برای استفاده است.

4. **پاسخ به پرسش:** با در نظر گرفتن تفسیر درست مانیتور واکسن و ارتباط دمای واکسن با ایمنی آن، پاسخ صحیح این است که **دایره بیرونی تیره‌تر از مربع داخلی** (گزینه [B]) نشان‌دهنده نامناسب بودن واکسن برای استفاده است.<|END_OF_TURN_TOKEN|>"""
cleaned_text = re.split(r'<\|CHATBOT_TOKEN\|>', text)[-1].strip().removesuffix('<|END_OF_TURN_TOKEN|>').strip()
print(cleaned_text)

**زنجیره افکار:**

1. **تفسیر درست مانیتور واکسن:** متخصص پزشکی درست اشاره می‌کند که مانیتور واکسن، مانند یک نشانگر حرارتی (vvm)، سلامت واکسن را بر اساس دمای آن ارزیابی می‌کند، نه اثرات بالینی.

2. **ارتباط دمای واکسن با ایمنی:**  ما باید به یاد داشته باشیم که واکسن OPV باید در دمای مناسب نگهداری و تزریق شود. اگر مانیتور نشان دهد که واکسن در معرض دماهای نامناسب قرار گرفته است (مثلاً دایره بیرونی تیره‌تر از مربع داخلی باشد)، آنگاه واکسن فاسد یا نامناسب برای استفاده است.

3. **تفسیر علائم دمایی:** 
   - **دایره بیرونی روشن‌تر از مربع داخلی:** نشان‌دهنده دمای مناسب و سالم بودن واکسن است.
   - **دایره بیرونی تیره‌تر از مربع داخلی:** نشان‌دهنده آسیب حرارتی و نامناسب بودن واکسن برای استفاده است.

4. **پاسخ به پرسش:** با در نظر گرفتن تفسیر درست مانیتور واکسن و ارتباط دمای واکسن با ایمنی آن، پاسخ صحیح این است که **دایره بیرونی تیره‌تر از مربع داخلی** (گزینه [B]) نشان‌دهنده نامناسب بودن واکسن برای استفاده است.
